# Grid Analysis for one Day Data 
This is the analysis for the data of one day using folium and pandas library and plot the Choropleth graph<br>
reference: https://github.com/nancygaooo/Visualization-Practice/blob/master/Folium%20Visualization%20Examples.ipynb<br>
reference: https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data<br>
reference(Mandarian):https://www.zhihu.com/question/33783546<br>
reference:https://www.jpytr.com/post/analysinggeographicdatawithfolium/

In [14]:
#import the required libriary used for the analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json as js
import ast
import folium
import re
import networkx as nx
from datetime import datetime

In [15]:
#open places file and handle the issue of single quotes
with open(r"places.txt", encoding="utf8") as corona_file:
    pl=[]
    line= corona_file.readlines()
    for row in range(len(line)):
        temp = ast.literal_eval(line[row])
        pl.append(temp)

    pls_df=pd.DataFrame(pl)

In [16]:
#helper function to calculate the average and apply them to all
def average(number1, number2):
    return (number1 + number2) / 2.0

#compute the average data of latitude and lontitude 
pls_df['Lontitude'] = pls_df.apply(lambda x: average (x['geo']['bbox'][0],x['geo']['bbox'][2]),axis = 1)
pls_df['Latitude'] = pls_df.apply(lambda x: average (x['geo']['bbox'][1],x['geo']['bbox'][3]),axis = 1)
pls_df

,full_name,place_type,id,country_code,country,geo,name,Lontitude,Latitude
0,"Düsseldorf, Allemagne",city,b7d3c12268abd20e,DE,Allemagne,"{'type': 'Feature', 'bbox': [6.688728, 51.1243...",Düsseldorf,6.814308,51.238447
1,"Manhattan, NY",city,01a9a39529b27f36,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-74.026675, 40.68...",Manhattan,-73.968542,40.780709
2,"Connecticut, USA",admin,e86b380cfefcced5,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-73.727776, 40.95...",Connecticut,-72.757385,41.500753
3,"नई दिल्ली, भारत",city,317fcc4b21a604d5,IN,Inde,"{'type': 'Feature', 'bbox': [76.84252, 28.3976...",नई दिल्ली,77.095086,28.638489
4,"Albuquerque, NM",city,813a485b26b8dae2,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-106.7916912, 35....",Albuquerque,-106.632718,35.117003
...,...,...,...,...,...,...,...,...,...
118818,sri ramanujar temple,poi,07d9e8a713086002,IN,Inde,"{'type': 'Feature', 'bbox': [79.948974609375, ...",sri ramanujar temple,79.948975,12.968259
118819,somewhere in India,poi,0909a45fdf54c001,IN,Inde,"{'type': 'Feature', 'bbox': [77.846767, 29.674...",somewhere in India,77.846767,29.674320
118820,Muthara,poi,07d9cc9aef083003,KE,Kenya,"{'type': 'Feature', 'bbox': [37.79895620871095...",Muthara,37.798956,0.216096
118821,Pensnett Doctors,poi,07d9cb7e4d885000,GB,Royaume-Uni,"{'type': 'Feature', 'bbox': [-2.131352, 52.500...",Pensnett Doctors,-2.131352,52.500561


In [17]:
#open the data file and handle the issue of single quotes
with open(r"D:\covid\corona_tweets_01_data.txt", encoding="utf8") as corona_file:
    tw=[]
    line= corona_file.readlines()
    for row in range(len(line)):
        temp = ast.literal_eval(line[row])
        tw.append(temp)
    tws_df=pd.DataFrame(tw)

In [18]:
#define helper function to clear the no places situation
def geo_nan(df):
    if isinstance(df.loc['geo'], float):
        return False
    if 'place_id' in df['geo']:
        return True
    return False

tws_df['geo_nan'] = tws_df.apply(geo_nan, axis=1)
tws_df = tws_df[tws_df['geo_nan'] == True]

#merge the two dataframe with the geo-id
tws_df['geo_id'] = tws_df.apply(lambda x: x['geo']['place_id'],axis = 1)
tws_merge = pd.merge(tws_df,pls_df,how = 'inner',left_on = 'geo_id',right_on = 'id')
tws_merge

,author_id,lang,public_metrics,entities,attachments,referenced_tweets,id_x,context_annotations,source,created_at,...,geo_id,full_name,place_type,id_y,country_code,country,geo_y,name,Lontitude,Latitude
0,2306150944,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,NaN,NaN,1240727823830130688,"[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter for iPhone,2020-03-19T19:52:19.000Z,...,00e55e2b4c491c5f,"Abuja, Nigeria",city,00e55e2b4c491c5f,NG,Nigeria,"{'type': 'Feature', 'bbox': [7.3045544, 8.9618...",Abuja,7.424094,9.073664
1,801892591285506053,en,"{'retweet_count': 0, 'reply_count': 2, 'like_c...","{'hashtags': [{'start': 123, 'end': 136, 'tag'...",NaN,NaN,1240731240719884288,"[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter for Android,2020-03-19T20:05:54.000Z,...,00e55e2b4c491c5f,"Abuja, Nigeria",city,00e55e2b4c491c5f,NG,Nigeria,"{'type': 'Feature', 'bbox': [7.3045544, 8.9618...",Abuja,7.424094,9.073664
2,159033468,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'annotations': [{'start': 28, 'end': 39, 'pro...",NaN,"[{'type': 'quoted', 'id': '1240711779384094721'}]",1240732480883302400,"[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter for Android,2020-03-19T20:10:49.000Z,...,00e55e2b4c491c5f,"Abuja, Nigeria",city,00e55e2b4c491c5f,NG,Nigeria,"{'type': 'Feature', 'bbox': [7.3045544, 8.9618...",Abuja,7.424094,9.073664
3,702160979,en,"{'retweet_count': 1, 'reply_count': 2, 'like_c...","{'annotations': [{'start': 29, 'end': 35, 'pro...",NaN,NaN,1240732545186181121,"[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter for Android,2020-03-19T20:11:05.000Z,...,00e55e2b4c491c5f,"Abuja, Nigeria",city,00e55e2b4c491c5f,NG,Nigeria,"{'type': 'Feature', 'bbox': [7.3045544, 8.9618...",Abuja,7.424094,9.073664
4,139432369,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'annotations': [{'start': 4, 'end': 32, 'prob...",NaN,NaN,1240733560513925120,"[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter for Android,2020-03-19T20:15:07.000Z,...,00e55e2b4c491c5f,"Abuja, Nigeria",city,00e55e2b4c491c5f,NG,Nigeria,"{'type': 'Feature', 'bbox': [7.3045544, 8.9618...",Abuja,7.424094,9.073664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6320,8225542,en,"{'retweet_count': 0, 'reply_count': 1, 'like_c...","{'hashtags': [{'start': 98, 'end': 111, 'tag':...",NaN,"[{'type': 'replied_to', 'id': '124086118646945...",1240861187346010112,"[{'domain': {'id': '71', 'name': 'Video Game',...",Twitter for iPhone,2020-03-20T04:42:15.000Z,...,5d1bffd975c6ff73,"Louisville, CO",city,5d1bffd975c6ff73,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-105.1836432, 39....",Louisville,-105.141728,39.968900
6321,881240947161534464,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 0, 'end': 15, 'usernam...",NaN,"[{'type': 'replied_to', 'id': '124085517571742...",1240861261891592192,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter for iPhone,2020-03-20T04:42:33.000Z,...,3be6e3e32603c621,"Pelham Manor, NY",city,3be6e3e32603c621,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-73.823233, 40.88...",Pelham Manor,-73.805512,40.892955
6322,722193656422072320,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 0, 'end': 8, 'username...",NaN,"[{'type': 'replied_to', 'id': '124067486973954...",1240861323908407296,"[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter for Android,2020-03-20T04:42:48.000Z,...,a769d8f00b12878d,"Pacifica, CA",city,a769d8f00b12878d,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-122.515276, 37.5...",Pacifica,-122.479070,37.616095
6323,185974318,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,NaN,NaN,1240861332729204736,"[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter for iPhone,2020-03-20T04:42:50.000Z,...,1f9f17a2260b4ff8,"Purcell, OK",city,1f9f17a2260b4ff8,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-97.3957045, 34.9...",Purcell,-97.371815,35.015205


In [274]:
#define the boundary for the grid
lower_left_coordinates =[-125.000000,24.000000]
upper_right_coordinates =[-66.000000,50.000000]

#define the grind with the size of m_box * n_box
m_box = 70
n_box = 35

In [275]:
#filter by the geobox of the info
tws_filter = tws_merge[(tws_merge['Lontitude']>=lower_left_coordinates[0])&(tws_merge['Lontitude']<=upper_right_coordinates[0])&(tws_merge['Latitude']>=lower_left_coordinates[1])&(tws_merge['Latitude']<=upper_right_coordinates[1])]

In [276]:
#use a function to get which place grid box does the tweets belong to
def get_grid_identifier(df):
    x_index = 0
    y_index = 0
    
    x_step = (upper_right_coordinates[0]-lower_left_coordinates[0])/(m_box)
    y_step = (upper_right_coordinates[1]-lower_left_coordinates[1])/(n_box)
    
    while x_index < m_box:
        if lower_left_coordinates[0]+ (x_index+1)* x_step> df['Lontitude']:
            break
        x_index = x_index+1
    
        
    while y_index < n_box:
        if lower_left_coordinates[1]+ (y_index+1)* y_step> df['Latitude']:
            break
        y_index = y_index+1
        
    return str(x_index)+"x"+str(y_index)

tws_filter['identifier'] = tws_filter.apply(get_grid_identifier,axis = 1)
tws_filter

<ipython-input-276-5e9af28fd68a>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tws_filter['identifier'] = tws_filter.apply(get_grid_identifier,axis = 1)


,author_id,lang,public_metrics,entities,attachments,referenced_tweets,id_x,context_annotations,source,created_at,...,full_name,place_type,id_y,country_code,country,geo_y,name,Lontitude,Latitude,identifier
38,859271883279159297,en,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",NaN,NaN,NaN,1240727871095631872,"[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter for iPhone,2020-03-19T19:52:30.000Z,...,"California, USA",admin,fbd6d2f5a4e4a15e,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-124.482003, 32.5...",California,-119.306608,37.269176,6x17
39,781285034703491073,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 0, 'end': 16, 'usernam...",NaN,"[{'type': 'quoted', 'id': '1240701140141879298'}]",1240741265064341505,"[{'domain': {'id': '10', 'name': 'Person', 'de...",Twitter for iPhone,2020-03-19T20:45:44.000Z,...,"California, USA",admin,fbd6d2f5a4e4a15e,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-124.482003, 32.5...",California,-119.306608,37.269176,6x17
40,2841636924,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'annotations': [{'start': 57, 'end': 67, 'pro...",NaN,NaN,1240745322952790016,"[{'domain': {'id': '10', 'name': 'Person', 'de...",Twitter for iPhone,2020-03-19T21:01:51.000Z,...,"California, USA",admin,fbd6d2f5a4e4a15e,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-124.482003, 32.5...",California,-119.306608,37.269176,6x17
41,888083561743384576,en,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,NaN,NaN,1240753298140872705,"[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter for iPhone,2020-03-19T21:33:33.000Z,...,"California, USA",admin,fbd6d2f5a4e4a15e,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-124.482003, 32.5...",California,-119.306608,37.269176,6x17
42,2996168750,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'urls': [{'start': 12, 'end': 35, 'url': 'htt...",{'media_keys': ['3_1240759505693995010']},NaN,1240759511423397889,"[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter for iPhone,2020-03-19T21:58:14.000Z,...,"California, USA",admin,fbd6d2f5a4e4a15e,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-124.482003, 32.5...",California,-119.306608,37.269176,6x17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6319,120873512,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'urls': [{'start': 176, 'end': 199, 'url': 'h...",{'media_keys': ['16_1240861146535596032']},NaN,1240861157444988928,"[{'domain': {'id': '65', 'name': 'Interests an...",Twitter for iPhone,2020-03-20T04:42:08.000Z,...,"Montréal, Québec",city,36775d842cbec509,CA,Canada,"{'type': 'Feature', 'bbox': [-73.972965, 45.41...",Montréal,-73.723025,45.557830,60x29
6320,8225542,en,"{'retweet_count': 0, 'reply_count': 1, 'like_c...","{'hashtags': [{'start': 98, 'end': 111, 'tag':...",NaN,"[{'type': 'replied_to', 'id': '124086118646945...",1240861187346010112,"[{'domain': {'id': '71', 'name': 'Video Game',...",Twitter for iPhone,2020-03-20T04:42:15.000Z,...,"Louisville, CO",city,5d1bffd975c6ff73,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-105.1836432, 39....",Louisville,-105.141728,39.968900,23x21
6321,881240947161534464,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 0, 'end': 15, 'usernam...",NaN,"[{'type': 'replied_to', 'id': '124085517571742...",1240861261891592192,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter for iPhone,2020-03-20T04:42:33.000Z,...,"Pelham Manor, NY",city,3be6e3e32603c621,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-73.823233, 40.88...",Pelham Manor,-73.805512,40.892955,60x22
6322,722193656422072320,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'mentions': [{'start': 0, 'end': 8, 'username...",NaN,"[{'type': 'replied_to', 'id': '124067486973954...",1240861323908407296,"[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter for Android,2020-03-20T04:42:48.000Z,...,"Pacifica, CA",city,a769d8f00b12878d,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-122.515276,

In [ ]:
#count the number of tweets in different area
grind_data = pd.DataFrame(tws_filter['identifier'].value_counts())
grind_data.reset_index(inplace=True)
grind_data.rename(columns={'identifier':'Count'},inplace=True)

In [286]:
#define and initilize a world map centre at US
map_grid= folium.Map(location=[36.648693, -95.8974445],tiles="cartodbpositron", zoom_start=4)


In [287]:
#function to generate an m*n grid geo json
def get_geojson_grid(upper_right, lower_left, m,n):
    
    geo_json = {"type": "FeatureCollection",
                "properties":{
                    "lower_left": lower_left,
                    "upper_right": upper_right, 
                    "x_size": m,
                    "y_size":n
                },
                "features":[]}


    lat_steps = np.linspace(lower_left[1], upper_right[1], n+1)
    lon_steps = np.linspace(lower_left[0], upper_right[0], m+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]
    
    #mark the index
    x = 0
    y = 0

    for lat in lat_steps[:-1]:
        x = 0
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]
            
            #define the identifier name for it
            identity = str(x)+"x"+str(y)

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            grid_feature = {
                "type":"Feature",
                "properties":{
                    "x_index":x,
                    "y_index":y,
                    "identifier": identity
                },
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates],
                }
            }
            
            geo_json['features'].append(grid_feature)

            x = x + 1
        y = y+1

    return geo_json

In [288]:
#create the geojson of the grid
grid = get_geojson_grid(upper_right_coordinates, lower_left_coordinates , m_box,n_box)

In [ ]:
#create the choropleth 
folium.Choropleth(
    geo_data=grid,
    data=grind_data,
    columns=['index','Count'],
    key_on='feature.properties.identifier',
    #fill_color='red',
    fill_color='YlOrRd',
    fill_opacity=0.8,
    line_opacity=0.2,
    highlight=True,
    legend_name='The number of tweets in the area'
).add_to(map_grid)

map_grid